In [190]:
import importlib
import os
from models import *
from dataprocessor import *
def reload_custom_libs():
    import models
    import dataprocessor
    importlib.reload(models)
    importlib.reload(dataprocessor)

In [191]:
macd_timescales = [(8, 24), (16, 28), (32, 96)]
rtn_timescales = [1, 21, 63, 126, 252]
timesteps = 126

folder_path = 'data'
files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]
files = [file for file in files if file not in ("CC00.NYB.xlsx", "LB00.CME.xlsx", "ES00.CME.xlsx", "NQ00.CME.xlsx", "YM00.CBT.xlsx")]
data_list = []


data_list = process_data_list(files, macd_timescales, rtn_timescales, test = False)
target_set, labels, map = generate_tensors(data_list, timesteps, encoder_type = "one-hot", return_map=True)
context_set, _ = generate_tensors(data_list, timesteps, encoder_type = "one-hot", contain_next_day_rtn=True)

处理文件中。。: 100%|██████████| 12/12 [00:59<00:00,  4.93s/it]


one-hot 编码中...
one-hot 编码中...


In [192]:
batch_size = len(data_list)
data_set = data_binder(context_set, target_set, labels, batch_size=batch_size)
data_set.save(f"saved_data")

dataset = tf.data.Dataset.load("saved_data")

处理日期中..: 100%|██████████| 90358/90358 [39:33<00:00, 38.07it/s]


In [320]:
reload_custom_libs()
from models import *
from dataprocessor import *

hidden_dim = 64 # 128
warm_up = 63
target_std = tf.cast(5e-2, tf.float64)
features_len = len(macd_timescales) + len(rtn_timescales)
encoding_size = len(data_list) + 1
x_shape = (batch_size, timesteps, features_len)
s_shape = (batch_size, timesteps, encoding_size)

model = ModelWrapper(features_len, hidden_dim, encoding_size, num_heads = 4)
model.build((x_shape, s_shape))

for data in dataset.take(1):
    x_c, x_c_rtn, s_c, x, s, rtn_std, _, _ = data
    result = model(x_c, x_c_rtn, s_c, x, s)

In [323]:
def sharpe_loss(positions: tf.Tensor, rtn_std: tf.Tensor, target_std: float, warm_up: int):
    rtn_std = tf.transpose(rtn_std, perm = [0, 2, 1])
    target_std = tf.cast(target_std, tf.float64)
    practice_timesteps = positions.shape[1] - warm_up
    assets_daily_rtn = rtn_std[:, :, 0] / rtn_std[:, :, 1] * target_std * positions
    portfolio_daily_rtn = tf.reduce_sum(assets_daily_rtn, axis=0)
    portfolio_daily_rtn = portfolio_daily_rtn[-practice_timesteps:]
    mean = tf.math.reduce_mean(portfolio_daily_rtn)
    std = tf.math.reduce_variance(portfolio_daily_rtn) ** 0.5
    loss = - 252 ** 0.5 * mean / std
    return loss


def mle_loss(properties: tf.Tensor, rtn_std: tf.Tensor, target_std: float, warm_up: int):
    batch_size = properties.shape[0]
    time_steps = properties.shape[1]
    rtn_std = tf.transpose(rtn_std, perm = [0, 2, 1])
    mean, std = tf.squeeze(properties[:, :, 0])[-time_steps:], tf.squeeze(properties[:, :, 1])[-time_steps:]
    assets_daily_change_pct = rtn_std[:, :, 0] / rtn_std[:, :, 1] * target_std
    assets_daily_change_pct = assets_daily_change_pct[-time_steps:]
    print(mean)
    log_likelihood = -0.5 * tf.reduce_sum(
        tf.math.log(2.0 * tf.constant(np.pi, tf.float64) * std**2) + ((assets_daily_change_pct - mean)**2 / (std**2))
    )
    loss = log_likelihood * -1 / batch_size / (time_steps - warm_up)
    return loss 
    
    

# sharpe_loss(result[1], rtn_std, target_std, warm_up)
mle_loss(result[0], rtn_std, target_std, warm_up)

tf.Tensor(
[[ 0.12584984 -0.15766597 -0.31135345 ... -0.74588162 -0.69257611
  -0.6935032 ]
 [ 1.8968035   1.89382064  2.03731036 ...  2.01342392  2.02806735
   2.01587176]
 [ 1.81113768  1.93689287  2.02341795 ...  2.47515368  2.46943474
   2.53876543]
 ...
 [ 1.96044993  1.84132099  1.78567052 ...  1.20581222  1.23146439
   1.20552826]
 [ 0.09267747  0.06958246  0.08335954 ...  0.1735779   0.17149854
   0.2484284 ]
 [-0.41665697 -0.43353912 -0.43934435 ... -0.44027138 -0.42002133
  -0.36266631]], shape=(12, 126), dtype=float64)


<tf.Tensor: shape=(), dtype=float64, numpy=7279.862473172699>

<tf.Tensor: shape=(), dtype=float64, numpy=3.141592653589793>